In [1]:
#import tensorflow as tf
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import chi2
from math import floor
from sklearn.svm import LinearSVC
from joblib import Parallel, delayed

In [9]:
main_directory= '/home/a20114261/GeneInteractions/GeneInteractionsBN_Datasets/Balanced_Batches/censored_Erk_Target/'
TargetNode =X_test.keys()[0]

In [11]:
X_test = pd.read_csv("/home/a20114261/GeneInteractions/GeneInteractionsBN_Datasets/DataGeneCausality/Tt/Erk.csv",sep=";")
del X_test['CauseGene']
del X_test['EffectGene']
del X_test['Replicate']
del X_test['Treatment']
X_test['Pvalue']=list(map(lambda x: 1 if x<=0.05 else 0, X_test['Pvalue']))
for column in X_test:
    if (column != "Pvalue"):
        X_test[column] = list(map(lambda x: x+1 , X_test[column]))
x_test_heads = []
for i in range(0,len(X_test.keys())):
    max_v=1
    for row in X_test[X_test.keys()[i]]:
        if (row>max_v-1):
            max_v+=row-(max_v-1)
    save_i=i+1
    x_test_heads.append("Node"+str(save_i)+"@"+str(max_v))
X_test.columns = x_test_heads
Y_pred = X_test[X_test.keys()[0]]
TargetNode =X_test.keys()[0]
x_heads=x_test_heads

In [36]:
import os
from imblearn.metrics import geometric_mean_score

X_train=np.ndarray((0,len(X_test.keys())-1))
Y_train=np.ndarray((0,1))
for filename in os.listdir(main_directory):
    X = pd.read_csv(main_directory+'/'+filename)
    # each cell can be -1, 0 or 1
    #Pre-processing data
    #X['Pvalue']=list(map(lambda x: 1 if x<=0.01 else (0 if x>=0.5 else -1), X['Pvalue']))
    #print(main_directory+filename)
    X['Pvalue']=list(map(lambda x: 1 if x<=0.05 else 0, X['Pvalue']))
    del X['Unnamed: 0']
    del X['TrueIndex']
    del X['CauseGene']
    del X['EffectGene']
    del X['Replicate']
    del X['Treatment']
    for column in X:
        if (column != "Pvalue"):
            X[column] = list(map(lambda x: x+1 , X[column]))
    X.columns = x_heads        
    Y_train=np.append(X.as_matrix([TargetNode]),Y_train,axis=0)
    print(len(X.keys()))
    del X[TargetNode]
    print(len(X.keys()))
    X_train=np.append(X.as_matrix(X.keys()),X_train,axis=0)
    x_train_heads = X.keys()

# making the new dataframe
X_train_df = pd.DataFrame(X_train,columns=x_train_heads)

541
540
541
540
541
540
541
540
541
540


In [37]:
X_train_df = pd.DataFrame(X_train,columns=x_train_heads)

In [38]:
X_train_df

,Node2@3,Node3@3,Node4@3,Node5@3,Node6@3,Node7@3,Node8@3,Node9@3,Node10@3,Node11@3,...,Node532@3,Node533@3,Node534@3,Node535@3,Node536@3,Node537@3,Node538@3,Node539@3,Node540@3,Node541@3
0,1.0,1.0,2.0,2.0,0.0,2.0,2.0,1.0,2.0,0.0,...,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0,2.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
2,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,2.0,2.0,2.0,1.0,2.0,0.0,2.0,0.0,2.0,2.0
3,0.0,2.0,2.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,...,1.0,2.0,0.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0
4,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5,2.0,0.0,2.0,0.0,2.0,0.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,2.0,1.0,...,2.0,2.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0
7,2.0,1.0,0.0,1.0,2.0,2.0,2.0,2.0,1.0,0.0,...,0.0,0.0,2.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0
8,0.0,0.0,0.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0
9,0.0,0.0,2.0,1.0,1.0,1.0,2.0,0.0,2.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,1.0


In [48]:
%%time
import warnings
warnings.filterwarnings("ignore")
cpc_test={}
cpc_test['cpc']=['Node341@3','Node96@3','Node358@3']
#cpc_test['filenames']=['batch_0.csv','batch_1.csv','batch_2.csv','batch_3.csv','batch_4.csv']
#cpc_test['filenames']=['batch_1.csv']
CandidateScore(cpc_test)


CPU times: user 1.04 s, sys: 169 ms, total: 1.21 s
Wall time: 2.59 s


In [42]:
def CandidateScore(PobDict):
    global X_test
    global x_heads
    global X_train_df
    global Y_train    
    # training and scoring
    clf = LinearSVC(random_state=0)
    X_cpc=np.ndarray((0,len(PobDict['cpc'])))
    #X_pred=np.ndarray((0,len(PobDict['cpc'])))
    # taking the whole balanced samples
    X_cpc=np.append(X_train_df.as_matrix(PobDict['cpc']),X_cpc,axis=0)
    X_pred=X_test.as_matrix(PobDict['cpc'])    
    clf.fit(X_cpc,Y_train)
    PobDict['score']=clf.score(X_pred,Y_pred)
#     print(PobDict['cpc'])
#     print(PobDict['score'])
    return PobDict